## Module Import 

In [ ]:
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import h5py
import re
import os
import time as t
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.python.keras.layers import Dense
np.set_printoptions(precision=8)
seed=98

print(tf. __version__)

2.7.0


In [ ]:
# change the directory to data location
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/PINN')

Mounted at /content/drive


In [ ]:
# Read the file
file = 'major_chemical_marker.h5'
f = h5py.File(file, 'r')

In [ ]:
# Get the feature and the label
# The dataset have larger points HHR region [data= Augumentation]
dataset = np.array(f.get('major_chemical_marker'))
print(dataset.shape)

(51966851, 5)


In [ ]:
# Separate the feature and the label
X = dataset[:,:-2]
Y = dataset[:,[-1]]
#print(X,Y.shape)

In [ ]:
#shuffle the dataset
X,Y = shuffle(X, Y, random_state=seed)
print(X, Y)

[[2.81994056e-04 3.19554703e-03 7.90773076e-04]
 [5.49053948e-05 1.46000320e-03 2.29734957e-04]
 [2.23566258e-05 9.57325610e-05 8.72315094e-03]
 ...
 [3.13814708e-05 1.23655904e-04 9.34661739e-03]
 [1.40960456e-05 8.52273733e-05 5.31098386e-03]
 [4.33951609e-05 1.39086868e-03 1.05819294e-04]] [[5.4436895e+09]
 [1.4833097e+09]
 [1.4732527e+09]
 ...
 [1.5068352e+09]
 [9.4112301e+08]
 [1.7772887e+09]]


In [ ]:
print(np.amin(Y))

14080775.0


In [ ]:
# create the normalization fit 
scale_X = MinMaxScaler()
scale_Y = MinMaxScaler()

# The fit function
print(scale_X.fit(X))
print(scale_Y.fit(Y))

MinMaxScaler()
MinMaxScaler()


In [ ]:
#test_train split 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(46770165, 3) (5196686, 3) (46770165, 1) (5196686, 1)


In [ ]:
# normalize the train data and label
X_train_n = scale_X.transform(X_train)
Y_train_n = scale_Y.transform(Y_train)
print(X_train_n)
print(Y_train_n)

[[0.19598909 0.09818207 0.3978071 ]
 [0.41989118 0.70258284 0.05572363]
 [0.03218278 0.12998788 0.02249385]
 ...
 [0.03479912 0.03093094 0.27022317]
 [0.9310691  0.8605492  0.2451434 ]
 [0.39254707 0.755976   0.05342871]]
[[0.10148043]
 [0.2790289 ]
 [0.00922257]
 ...
 [0.01803892]
 [0.9036713 ]
 [0.39911494]]


In [ ]:
# Create the chunk 
X_chunk = X_train_n[0:5000000,:]
Y_chunk = Y_train_n[0:5000000,:]
print(X_chunk, Y_chunk)

[[0.19598909 0.09818207 0.3978071 ]
 [0.41989118 0.70258284 0.05572363]
 [0.03218278 0.12998788 0.02249385]
 ...
 [0.05504358 0.2806627  0.00513289]
 [0.0472826  0.03428879 0.34884664]
 [0.31047314 0.4242381  0.18843953]] [[0.10148043]
 [0.2790289 ]
 [0.00922257]
 ...
 [0.07666439]
 [0.01682727]
 [0.4127905 ]]


In [ ]:
# Creaate the batch slices.
train_data = tf.data.Dataset.from_tensor_slices((X_chunk, Y_chunk))
train_data = train_data.batch(1024, drop_remainder=True)
print(train_data)
# test_data = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
# test_data = test_data.batch(1024, drop_remainder=True)

<BatchDataset shapes: ((1024, 3), (1024, 1)), types: (tf.float32, tf.float32)>


In [ ]:
# Get the pretrained model
# Read the file
file = 'nh_nh2_oh.h5'
model = tf.keras.models.load_model(file)

# Get summary
model.summary()


Model: "Heat_release_rate"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 h_layer1 (Dense)            (None, 40)                160       
                                                                 
 h_layer2 (Dense)            (None, 24)                984       
                                                                 
 h_layer3 (Dense)            (None, 24)                600       
                                                                 
 output_layers (Dense)       (None, 1)                 25        
                                                                 
Total params: 1,769
Trainable params: 1,769
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# The Ki the is respective exponents of the species massfraction and temperature, which could yeild us a perfect reconstruction of the HRR
#k1, k2, k3, k4 = tf.Variable([1.0], dtype=tf.float32),tf.Variable([-0.5], dtype=tf.float32),tf.Variable([-0.75], dtype=tf.float32),tf.Variable([1.0], dtype=tf.float32)
k1,k2,k3,k4 = tf.Variable(-1.0),tf.Variable(-1.0),tf.Variable(-0.75),tf.Variable(1.3)

In [ ]:
# The customized mean square error, taking the batch data at any instance
def customized_MSE(true, pred):
    MSE = tf.reduce_mean(tf.square(true-pred))
    return MSE
# The customized loss taking the non-linear equation as residual
def total_mse(MSE_NN, X, Y, k1, k2, k3, k4):# the MSE_NN is the mean squared error of the neural network 
    residual = tf.square(tf.math.log(Y) - (k1*tf.math.log(X[:,0:1]) + k2* tf.math.log(X[:,1:2]) + k3* tf.math.log(X[:,2:3]) +  tf.math.log(k4)))
    MSE_func = tf.reduce_mean(residual)
    total =  MSE_NN + MSE_func
    return total

In [ ]:
# Optimizer 
# optimizers = tf.keras.optimizers.SGD(learning_rate=0.3,momentum=0.8)
optimizers = tf.keras.optimizers.Adam(
    learning_rate=0.009, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')

In [ ]:
#Training with customized optimization technic (Automatic-Gradient)
epochs = 100
#lr_rate = 0.03

for epoch in range(epochs):
  print("Starting a new epoch",epoch+1)
  for step, (X_batch, Y_batch) in enumerate(train_data):
    with tf.GradientTape(persistent=True) as tape:
      Y_pred = model(X_batch)
      current_loss = customized_MSE(Y_batch,Y_pred)
      new_loss = total_mse(current_loss, X_batch, Y_batch, k1, k2, k3, k4)

    # The gradient check  
    gradient = tape.gradient(new_loss, [k1,k2,k3,k4])
    grad = tape.gradient(current_loss, model.trainable_variables)

    #Assign new variables to the model using optimizer isntead of sub assign
    optimizers.apply_gradients(zip(grad, model.trainable_variables))
    optimizers.apply_gradients(zip(gradient, [k1,k2,k3,k4]))

    #Update the Ki to reduce the residual
    # k1.assign_sub(gradient[0]*lr_rate)
    # k2.assign_sub(gradient[1]*lr_rate)
    # k3.assign_sub(gradient[2]*lr_rate)
    # k4.assign_sub(gradient[3]*lr_rate)
  
  print("k1:{} k2:{} k3:{} k4:{}".format(k1.numpy(),k2.numpy(),k3.numpy(),k4.numpy()))
  print("current_Loss: {} new_loss: {}".format(current_loss.numpy(), new_loss.numpy()))  

Starting a new epoch 1
k1:0.09803864359855652 k2:0.8401013016700745 k3:0.35241416096687317 k4:1.2079639434814453
current_Loss: 0.0007752791862003505 new_loss: 0.38055920600891113
Starting a new epoch 2
k1:-0.1107494980096817 k2:1.0677590370178223 k3:0.46663302183151245 k4:1.4899852275848389
current_Loss: 0.0007818022859282792 new_loss: 0.3777131140232086
Starting a new epoch 3
k1:-0.11111397296190262 k2:1.068498134613037 k3:0.4672434628009796 k4:1.4902453422546387
current_Loss: 0.0007546242559328675 new_loss: 0.37768539786338806
Starting a new epoch 4
k1:-0.11111590266227722 k2:1.0685018301010132 k3:0.46724528074264526 k4:1.49024498462677
current_Loss: 0.0007452393183484674 new_loss: 0.3776759207248688
Starting a new epoch 5
k1:-0.11111624538898468 k2:1.0685020685195923 k3:0.4672454297542572 k4:1.4902453422546387
current_Loss: 0.0007231591735035181 new_loss: 0.3776538372039795
Starting a new epoch 6
k1:-0.11111608147621155 k2:1.0685019493103027 k3:0.46724531054496765 k4:1.4902452230453